In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

1. `yfinance`: Esta es una biblioteca de Python que permite descargar datos históricos del mercado de Yahoo Finance. Se utiliza para obtener datos de acciones, opciones, futuros, etc.

2. `pandas`: Es una biblioteca de Python para la manipulación y análisis de datos. Proporciona estructuras de datos y funciones necesarias para manipular y analizar conjuntos de datos grandes y complejos.

3. `requests`: Esta biblioteca se utiliza para hacer solicitudes HTTP en Python. Puede ser utilizada para interactuar con servicios web.

4. `bs4 (BeautifulSoup)`: BeautifulSoup es una biblioteca de Python para extraer datos de archivos HTML y XML. Se utiliza para web scraping, que es el proceso de recopilación de datos de sitios web.

5. `plotly.graph_objects`: Esta es una biblioteca de Python para crear gráficos interactivos. Proporciona una interfaz de alto nivel para dibujar gráficos atractivos e informativos.

6. `plotly.subplots`: Este módulo de Plotly se utiliza para crear subtramas, que son gráficos que tienen múltiples gráficos en una sola figura.

En Python, puedes ignorar las advertencias utilizando el módulo de warnings. Puedes usar la función filterwarnings para filtrar o ignorar mensajes o categorías de advertencia específicos.


In [ ]:
# Ignore all warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)